In [ ]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
import google.auth
import plotly.express as px
import os

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./storage.json"
credentials, your_project_id = google.auth.default(
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)
bq = bigquery.Client()

In [ ]:
class performance_check_plant_id:
    def __init__(self,resource_category,filename,pred_time,check_start_date,check_end_date):
        self.resource_category = resource_category
        self.filename = filename
        self.pred_time = pred_time
        self.check_start_date = check_start_date
        self.check_end_date = check_end_date
        
    def read_result_file(self):
        result_df = pd.read_csv("../"+self.resource_category+"_연구/결과파일/"+self.filename+self.pred_time+'.csv',index_col=0)
        result_df = result_df[(result_df.base_date>=self.check_start_date)&(result_df.base_date<=self.check_end_date)].reset_index(drop=True)
        result_df['gen_hour'] = result_df['pv_amount'] / result_df['kpx_capacity']
        result_df['gen_hour_pred'] = result_df['pv_amount_prediction'] / result_df['kpx_capacity']
        return result_df
    
    def show_plotly_plant_id_result(self,plant_ids,result_df):
        if not os.path.exists('./plotly_image_plant_ids/'+self.resource_category):
            os.makedirs('./plotly_image_plant_ids/'+self.resource_category)
        for plant_id in plant_ids:
            result_df_plant_id = result_df[result_df.plant_id==plant_id].reset_index(drop=True)
            fig = px.line(result_df_plant_id,x='base_time',y=['gen_hour','gen_hour_pred'],title=str(plant_id))
            fig.write_html("./plotly_image_plant_ids/"+self.resource_category+'/'+str(plant_id)+'.html')
    
    @staticmethod
    def get_nmae(result_merge):
        tot_nmae = np.mean(result_merge.nmae[result_merge.use>=0.1])
        tc= np.sum(result_merge.pv_amount[result_merge.use>=0.1])*4
        c4 =np.sum(result_merge.pv_amount[(result_merge.use>=0.1)&(result_merge.nmae<=0.06)])*4
        c3 =np.sum(result_merge.pv_amount[(result_merge.use>=0.1)&(result_merge.nmae>0.06)&(result_merge.nmae<=0.08)])*3
        tcc = (c4+c3)/tc
        return tot_nmae
    
    @staticmethod
    def monthly_result_report(monthly_result_df):
        monthly_plant_result = []
        for base_date in monthly_result_df.base_date.unique():
            fslc003_pred0_by_date = monthly_result_df[monthly_result_df.base_date==base_date].reset_index(drop=True)
            for plant_id in fslc003_pred0_by_date.plant_id.unique():
                fslc003_pred0_by_date_pid = fslc003_pred0_by_date[fslc003_pred0_by_date.plant_id==plant_id].reset_index(drop=True)
                monthly_plant_result.append([base_date,plant_id,np.round(performance_check_plant_id.get_nmae(fslc003_pred0_by_date_pid),4)])
        monthly_plant_result_df = pd.DataFrame(monthly_plant_result,columns=['base_date','plant_id','nmae'])
        monthly_plant_result_df_nmae_count = monthly_plant_result_df.pivot_table(index=['plant_id'],values='nmae',aggfunc= lambda x:(x>=0.1).sum()).reset_index()
        monthly_plant_result_df_nmae_count = monthly_plant_result_df_nmae_count.sort_values(by=['nmae']).reset_index(drop=True)
        monthly_plant_result_df_nmae_count = monthly_plant_result_df_nmae_count.rename(columns={'nmae':'nmae_count'})
        return monthly_plant_result_df_nmae_count,tuple(monthly_plant_result_df_nmae_count.plant_id.unique())
    
    @staticmethod
    def plant_information(monthly_result_df,plant_ids):
        plant_informaiton_query = f"""
            WITH A as (
                    SELECT CAST(id as string)plant_id, name as plant_name
                    FROM sc-data-mart.mysql_tables.plant_master where id in {plant_ids}
                )
            select A.*,B.pv_capacity,B.station_id,B.icsr_station_id,B.icsr_distance_km,B.plant_latitude,B.plant_longitude from A
            inner join sc-data-mart.feature_store.plant_mapping_table as B on A.plant_id = B.plant_id where station_id = icsr_station_id
        """
        plant_information_df = bq.query(plant_informaiton_query).to_dataframe()
        monthly_result_df.plant_id = monthly_result_df.plant_id.astype(str)
        monthly_result_df = pd.merge(monthly_result_df,plant_information_df,on=['plant_id'])
        return monthly_result_df
        

In [ ]:
performance_check_plant_id = performance_check_plant_id(
    resource_category = "fSLC003",
    filename = "result_ldaps_",
    pred_time='1',
    check_start_date = "2022-11-01",
    check_end_date = "2022-11-30",
)

In [ ]:
result_df = performance_check_plant_id.read_result_file()
monthly_result,plant_id_tuple = performance_check_plant_id.monthly_result_report(result_df)
performance_result_plant_id = performance_check_plant_id.plant_information(monthly_result,plant_id_tuple)
## plant_id 별 결과 요약
performance_result_plant_id

In [ ]:
performance_check_plant_id.show_plotly_plant_id_result(plant_id_tuple,result_df)